In [ ]:
# Installing simpletransformers here 
# because of conflicting transformer version needed by GoEmotions
!pip install simpletransformers

In [ ]:
# Activates CUDA for GPU use on T5 model prediction generation
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'
torch.multiprocessing.set_sharing_strategy('file_system')

In [ ]:
# Allows user input for model used for paraphrase generation based on:
# 1. Data set used to train model
# 2. Training type of model
# 3. Epochs used to train model
# Allows user to specify evaluation data set 
model_dataset = input('Dataset used to train model (twit0.825, mix, or combined): ')
training_type = input('Training type of model: ')
epochs = int(input('Number of epochs trained on model'))
model_name = f"{training_type}-{model_dataset}-{epochs}epochs"

testing_set = input('Dataset for prediction generation: ')

if training_type.lower() == 'regular':
    train_set = 'training'
    eval_set = 'testing'
elif training_type.lower() == 'limited':
    train_set = 'testing'
    eval_set = 'training'
else:
    print('Please enter a valid training type')

In [ ]:
# Prediction generation using specified model and evaluation set 
# Type of evaluation set (training or testing) is opposite of type used to train the specificed model
import pandas as pd
import os
from simpletransformers.t5 import T5Model

print(f'---- Generating Predictions for Model: {model_name}  on Dataset: {testing_set}----')

# General parameters for prediction generation
model_args = {
    "overwrite_output_dir": True,
    "max_seq_length": 196,
    "eval_batch_size": 32,
    "num_train_epochs": 1,
    "use_multiprocessing": False,
    "num_beams": None,
    "do_sample": True,
    "max_length": 50,
    "top_k": 50,
    "top_p": 0.95,
    "num_return_sequences": 3,
}

# Imports model and evaluation setfrom current directory
model = T5Model("t5", model_name, args=model_args)
df = pd.read_csv(f'emotion-labeled-data/{model_dataset}/{testing_set}-{eval_set}_t5.tsv', sep="\t").astype(str)

# Generates statements used for predictions
to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(df["prefix"].tolist(), df["input_text"].tolist())
]

# Generates the predictions and adds them to the dataframe
preds = model.predict(to_predict)
preds = [pred[0] for pred in preds]
df["predictions"] = preds
df = df.drop(['Unnamed: 0'], axis=1)

# Ensures directory exists for save location
path = f'predictions-data/{model_name}'
if not os.path.exists(path):
    os.mkdir(path)

# Saves file with predictions to a new directory for the scoring stage
df.to_csv(f'predictions-data/{model_name}/{testing_set}-predictions.tsv', sep="\t")